In this file, we load the different images from Airyscan and STED images (make sure that they are present in the folder, since they are uploaded outside of github).

Then we perfom different preprocessing steps using a string of different steps to apply. the filenames inherit these names. 
Then cubical persistence homology is computed using the superlevelsets by using the negative values for the images, since the standard algorithms in Gudhi use sublevelset-evolution for computations. 

To ensure a smaller filesize in the repository, we split large files along the dimension of the input images (either 2 or 3). If the files are still too large, we split them further; use "read_persistence_files" to gather all files for a specific microscope as well as a specific preprocessing.

For further information about the different preprocessing steps of the images, see `additionalAnalysis1_preprocessing.ipynb`.

In [ ]:
import numpy as np
import pandas as pd

import gudhi as gd
from sklearn.preprocessing import MinMaxScaler
from scipy.ndimage import gaussian_filter

from pathlib import Path
from tqdm.notebook import tqdm

from src.inputreader import read_segmented_images, persistence_writer_ensure_filesize

In [ ]:
data_input = Path('data_segmented')
input_airyscan = data_input / 'Airyscan'
input_sted = data_input / 'STED'

data_pers = Path('data_processed')
pers_sted = data_pers / 'persistence_sted'
pers_airyscan = data_pers / 'persistence_airyscan'

Preprocessing functions:

Given an image img and minval = np.min(img), we do the following preprocessing steps for the name

1) 'clip': clip the valus to `q05 = np.quantile(img > np.min, 0.05)` and `q95 = np.quantile(img > np.min, 0.95)`, i.e. all values lower than q05 are set to q05 and all values larger than q95 are set to q95.
2) 'minmax' (first): Scale the imaage values such that they start at 0 and end at 1
3) 'gaussian{2/3/4}{a/c}': Do a gaussian smoothing using sigma=[1,1,1] (for 3d) and sigma=[1,1] (in 2d) for (a) and sigma=[1, sigmapixel[1]/sigmapixel[0], sigmapixel[1]/sigmapixel[0]] for (b) with truncate=2/3/4
4) 'minmax': although not needed after gaussian, we still make it exact with that
5) 'mask0': reset certain values to 0 from one of the earlier steps (TO BE DELETED)



In [ ]:
# preprocessing functions


preprocessings = ['clip_minmax_gaussian2c_minmax']
# 'clip_minmax_gaussian2a_mask0',
# 'clip_minmax_gaussian2c_mask0',
# 'clip_minmax_gaussian4a_minmax_mask0',
# 'clip_minmax_gaussian4c_minmax_mask0',
# 'clip_minmax_gaussian4a_mask0',
# 'clip_minmax_gaussian4c_mask0',
# 'clip_gaussian2a_minmax_mask0',
# 'clip_gaussian2c_minmax_mask0',
# 'clip_gaussian4a_minmax_mask0',
# 'clip_gaussian4c_minma_mask0',
# 'clip_minmax_gaussian3a_minmax_mask0',
# 'clip_minmax_gaussian3c_minmax_mask0',
# 'clip_minmax_gaussian3a_mask0',
# 'clip_minmax_gaussian3c_mask0'
# 'clip_gaussian3a_minmax_mask0',
# 'clip_gaussian3c_minmax_mask0',

# STED

In [ ]:
# get the metadata for the original files as well as the segmented files
df_sted_metadata = pd.read_csv(data_input/ 'sted_df_metadata.csv', comment='#')

masks, bounding_boxes, labels_str, labels, npzfiles = \
    read_segmented_images(input_sted, microscope='sted', replace_nan_with=0)

df_labels = pd.DataFrame(labels_str, columns=['labels_str'])
df_labels.loc[:, 'labels'] = labels
df_labels.loc[:, 'id'] = np.arange(len(labels))
df_labels.loc[:, 'microscope'] = 'sted'
df_labels.loc[:, 'filename'] = [f.name for f in npzfiles]
df_labels.to_csv(Path(data_pers, 'labels_persistence_sted.csv'),
                 index=False)

# get the physical pixel sizes for each image
for filename in npzfiles:
    assert len(df_sted_metadata.loc[df_sted_metadata['segmented_filename'] == filename.name, :]) == 1
pixelsizes = [df_sted_metadata.loc[df_sted_metadata['segmented_filename'] == filename.name,
                                   ['pixel_size_z', 'pixel_size_x', 'pixel_size_y']]\
                                    .values[0] for filename in npzfiles]
pixelsizes = np.array(pixelsizes)

In [ ]:
sigma = 1

for preproc in tqdm(preprocessings):
    # if Path(pers_sted/f'persistence_sted_{preproc}.npz').exists():
    #     continue
    # clipped_images = []
    pers_all = {2: [[], []], 3: [[], [], []]}
    for i, mask_loop in tqdm(enumerate(masks)):
        mask = mask_loop.astype(np.float64)
        if np.any(np.isnan(mask)):
            assert np.nanmin(mask) == 0
            mask[np.isnan(mask)] = 0
        mask_org = mask.copy()

        if preproc != 'raw':
            quant05 = np.nanquantile(mask[mask > np.min(mask)], 0.05)
            quant95 = np.nanquantile(mask[mask > np.min(mask)], 0.95)
            mask = np.clip(mask, quant05, quant95)
        
        if 'clip_minmax' in preproc:
            mask = MinMaxScaler().fit_transform(mask.reshape(-1, 1)).reshape(mask.shape)
        
        if 'gaussian' in preproc and ('a_minmax' in preproc or 'a_mask0' in preproc or preproc.endswith('a')):
            sigma_pixels = 1
        elif 'gaussian' in preproc and ('b_minmax' in preproc or 'b_mask0' in preproc or preproc.endswith('b')):
            # set the sigmas such that pixel_x and pixel_y are 1
            sigma_pixels = pixelsizes[i].copy()
            # x and y resolution should be the same
            assert sigma_pixels[1] == sigma_pixels[2]
            sigma_pixels /= sigma_pixels[1]
        elif 'gaussian' in preproc and ('c_minmax' in preproc or 'c_mask0' in preproc or preproc.endswith('c')):
            # set the sigmas such that pixel_z are 1
            sigma_pixels = pixelsizes[i].copy()
            # x and y resolution should be the same
            assert sigma_pixels[1] == sigma_pixels[2]
            sigma_pixels /= sigma_pixels[0]
        
        if 'gaussian2' in preproc:
            mask = gaussian_filter(mask, sigma=sigma_pixels, truncate=2, mode='constant', cval=0.0)
        elif 'gaussian3' in preproc:
            mask = gaussian_filter(mask, sigma=sigma_pixels, truncate=3, mode='constant', cval=0.0)
        elif 'gaussian4' in preproc:
            mask = gaussian_filter(mask, sigma=sigma_pixels, truncate=4, mode='constant', cval=0.0)
        
        if preproc.endswith('minmax') or preproc.endswith('minmax_mask0'):
            mask = MinMaxScaler().fit_transform(mask.reshape(-1, 1)).reshape(mask.shape)
        if 'mask0' in preproc:
            mask[mask_org == 0] = 0
        clipped_images = mask.copy()

        for max_dim in [2, 3]:
            if max_dim == 2:
                cc = gd.CubicalComplex(top_dimensional_cells= -np.max(clipped_images.astype(np.float64), axis=0))
            else:
                cc = gd.CubicalComplex(top_dimensional_cells= -clipped_images.astype(np.float64))
            cc.compute_persistence()

            for dimi in range(max_dim):
                persistence = cc.persistence_intervals_in_dimension(dimi)
                pers_all[max_dim][dimi].append(persistence.copy())

    pers_save = {'labels': labels_str}
    for key in pers_all.keys():
        for dim in range(len(pers_all[key])):
            for i in range(len(pers_all[key][dim])):
                newkey = f'pers-{key}_dim-{dim}_i-{i:04d}'
                assert newkey not in pers_save
                pers_save[newkey] = pers_all[key][dim][i].copy()
                if np.shape(pers_all[key][dim][i])[0] < 10:
                    print(newkey, key, dim, i, np.shape(pers_all[key][dim][i])[0])

    filepath = pers_sted/f'persistence_sted_{preproc}.npz'
    np.savez_compressed(filepath, **pers_save)

    # check the filesize and delete it if it is too large
    # this is done to ensure the non LFS filesize of github
    if filepath.is_file() and filepath.stat().st_size / (1024*1024) < 95:
        persistence_writer_ensure_filesize(filepath,
            maxfilesize=95, lowerlimit=90,
            split_names='splitfiles',
            pers2d3d=['pers-2', 'pers-3'])
        filepath.unlink()
    del pers_save
    del pers_all

# Airyscan

In [ ]:
df_airy_metadata = pd.read_csv(data_input / 'airyscan_df_metadata.csv')
masks, bounding_boxes, labels_str, labels, npzfiles = read_segmented_images(input_airyscan, microscope='airyscan')

df_labels = pd.DataFrame(labels_str, columns=['labels_str'])
df_labels.loc[:, 'labels'] = labels
df_labels.loc[:, 'id'] = np.arange(len(labels))
df_labels.loc[:, 'microscope'] = 'airyscan'
df_labels.loc[:, 'filename'] = [f.name for f in npzfiles]

df_labels.to_csv(Path(data_pers, 'labels_persistence_airyscan.csv'),
                 index=False)

pixelsizes = [df_airy_metadata.loc[df_airy_metadata['filename_segmented'] == filename.name,
              ['pixel_size_z', 'pixel_size_x', 'pixel_size_y']]\
              .values[0] for filename in npzfiles]
pixelsizes = np.array(pixelsizes)

assert np.all(pixelsizes[:, 1] == pixelsizes[:, 2])

In [ ]:
sigma = 1

for preproc in tqdm(preprocessings):
    # if Path(pers_sted/f'persistence_sted_{preproc}.npz').exists():
    #     continue
    # clipped_images = []
    pers_all = {2: [[], []], 3: [[], [], []]}
    for i, mask_loop in tqdm(enumerate(masks)):
        mask = mask_loop.astype(np.float64)
        if np.any(np.isnan(mask)):
            assert np.nanmin(mask) == 0
            mask[np.isnan(mask)] = 0
        mask_org = mask.copy()

        if preproc != 'raw':
            quant05 = np.nanquantile(mask[mask > np.min(mask)], 0.05)
            quant95 = np.nanquantile(mask[mask > np.min(mask)], 0.95)
            mask = np.clip(mask, quant05, quant95)
        
        if 'clip_minmax' in preproc:
            mask = MinMaxScaler().fit_transform(mask.reshape(-1, 1)).reshape(mask.shape)
        
        if 'gaussian' in preproc and ('a_minmax' in preproc or 'a_mask0' in preproc or preproc.endswith('a')):
            sigma_pixels = 1
        elif 'gaussian' in preproc and ('b_minmax' in preproc or 'b_mask0' in preproc or preproc.endswith('b')):
            # set the sigmas such that pixel_x and pixel_y are 1
            sigma_pixels = pixelsizes[i].copy()
            # x and y resolution should be the same
            assert sigma_pixels[1] == sigma_pixels[2]
            sigma_pixels /= sigma_pixels[1]
        elif 'gaussian' in preproc and ('c_minmax' in preproc or 'c_mask0' in preproc or preproc.endswith('c')):
            # set the sigmas such that pixel_z are 1
            sigma_pixels = pixelsizes[i].copy()
            # x and y resolution should be the same
            assert sigma_pixels[1] == sigma_pixels[2]
            sigma_pixels /= sigma_pixels[0]
        
        if 'gaussian2' in preproc:
            mask = gaussian_filter(mask, sigma=sigma_pixels, truncate=2, mode='constant', cval=0.0)
        elif 'gaussian3' in preproc:
            mask = gaussian_filter(mask, sigma=sigma_pixels, truncate=3, mode='constant', cval=0.0)
        elif 'gaussian4' in preproc:
            mask = gaussian_filter(mask, sigma=sigma_pixels, truncate=4, mode='constant', cval=0.0)
        
        if preproc.endswith('minmax') or preproc.endswith('minmax_mask0'):
            mask = MinMaxScaler().fit_transform(mask.reshape(-1, 1)).reshape(mask.shape)
        if 'mask0' in preproc:
            mask[mask_org == 0] = 0
        clipped_images = mask.copy()

        for max_dim in [2, 3]:
            if max_dim == 2:
                cc = gd.CubicalComplex(top_dimensional_cells= -np.max(clipped_images.astype(np.float64), axis=0))
            else:
                cc = gd.CubicalComplex(top_dimensional_cells= -clipped_images.astype(np.float64))
            cc.compute_persistence()

            for dimi in range(max_dim):
                persistence = cc.persistence_intervals_in_dimension(dimi)
                pers_all[max_dim][dimi].append(persistence.copy())

    pers_save = {'labels': labels_str}
    for key in pers_all.keys():
        for dim in range(len(pers_all[key])):
            for i in range(len(pers_all[key][dim])):
                newkey = f'pers-{key}_dim-{dim}_i-{i:04d}'
                assert newkey not in pers_save
                pers_save[newkey] = pers_all[key][dim][i].copy()
                if np.shape(pers_all[key][dim][i])[0] < 10:
                    print(newkey, key, dim, i, np.shape(pers_all[key][dim][i])[0])

    filepath = pers_airyscan / f'persistence_airyscan_{preproc}.npz'
    np.savez_compressed(filepath, **pers_save)

    # check the filesize and delete it if it is too large
    # this is done to ensure the non LFS filesize of github
    if filepath.is_file() and filepath.stat().st_size / (1024*1024) < 95:
        persistence_writer_ensure_filesize(filepath,
            maxfilesize=95, lowerlimit=90,
            split_names='splitfiles',
            pers2d3d=['pers-2', 'pers-3'])
        filepath.unlink()
    del pers_save
    del pers_all